[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/LorBordin/bodynet/blob/master/dev_utils/training.ipynb)

In [2]:
import sys
sys.path.append("../")

In [3]:
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from tensorflow import keras
from imutils import paths
import tensorflow as tf
import numpy as np
import cv2
import os

from bodypose.training.metrics import avgMDE_2D, avgMDE_2D_RAW, Accuracy
from bodypose.training.metrics import RegressionLoss2D, AuxiliaryLoss
from bodypose.training.preprocessing import load_TFRecords_dataset  
from bodypose.training.preprocessing import augmentations
from bodypose.training.architecture import MoveNet
import config as cfg

In [2]:
INPUT_SHAPE = (224, 224, 3)
STRIDES = (32, 16, 8, 4)
NUM_KPTS = cfg.N_KPTS

GRID_SIZE = INPUT_SHAPE[0] // STRIDES[-1]

MODEL_PATH = f"./saved_models/movenet_{INPUT_SHAPE[0]}.models"


# 1. Load the Dataset

In [ ]:
augmentations = [
    augmentations.VerticalShift(max_shift_range=.15),
    augmentations.HorizontalShift(max_shift_range=.15),
    augmentations.HorizontalFlip(probability=.5)
    ]

In [3]:
filePaths = list(paths.list_files("./dataset/tfrecords/coco/validation"))
np.random.shuffle(filePaths)
print(f"[INFO] Found {len(filePaths)} TFRecords.")

train_paths = filePaths[:-2]
valid_path = filePaths[-2:]

train_ds = load_TFRecords_dataset(
    filePaths=train_paths, 
    batch_size = 32,
    target_size = INPUT_SHAPE[:2],
    grid_dim = GRID_SIZE,
    augmentations = augmentations,
    roi_thresh = 0.0
    )

val_ds = load_TFRecords_dataset(
    filePaths=valid_path, 
    batch_size = 32,
    target_size = INPUT_SHAPE[:2],
    grid_dim = GRID_SIZE,
    augmentations = [],
    roi_thresh = 0.0
    )

[INFO] Found 10 TFRecords.


2022-10-27 15:42:25.031997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
for img, (y1, y2) in train_ds.take(1):
    print(img.shape)
    print(y1.shape)
    print(y2.shape)

(32, 224, 224, 3)
(32, 17, 5)
(32, 3136, 18)


# 2. Load the model

In [5]:
model = MoveNet(
    input_shape = INPUT_SHAPE, 
    strides = STRIDES, 
    num_joints = NUM_KPTS, 
    alpha = .25, 
    use_depthwise = True
    )

img = (np.random.uniform(
    0, 255, (1,) + INPUT_SHAPE
    ).astype("uint8") / 255).astype(np.float32)

%timeit model(img)


112 ms ± 639 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
adam = keras.optimizers.Adam()
moving_avg_opt = tfa.optimizers.MovingAverage(adam)

def scheduler(epoch, lr):
    return  lr
    #if epoch < 10:
    #    return lr
    #else:
    #    return lr * tf.math.exp(-0.1)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        MODEL_PATH,
        monitor = "val_avgMDE_2D",
        save_best_only = True,
        save_weights_only = True,
        initial_value_threshold=None,
        ),
    keras.callbacks.LearningRateScheduler(
        scheduler
        ),
    tfa.callbacks.AverageModelCheckpoint(
        filepath=MODEL_PATH, 
        update_weights=True
        )
]

model.compile(
    optimizer = moving_avg_opt,
    loss = [RegressionLoss2D,AuxiliaryLoss],
    loss_weights = [1., 1.],
    metrics = [Accuracy, avgMDE_2D_RAW, avgMDE_2D],
)

In [14]:
model.fit(train_ds.take(2), epochs=1)

2/2 [==============================] - 18s 2s/step - loss: 647562.0000 - output_1_loss: 212.9706 - output_2_loss: 647349.0000 - output_1_Accuracy: 0.2996 - output_1_avgMDE_2D_RAW: 0.7127 - output_1_avgMDE_2D: 0.4089 - output_2_Accuracy: 0.0909 - output_2_avgMDE_2D_RAW: 0.4821 - output_2_avgMDE_2D: 0.4296
